In [1]:
from llama_cpp import Llama
from pathlib import Path
import glob
import time

In [2]:
MODEL_DIR = r"C:\models\bggpt-2.6b"   # <-- change if different
PATTERN = r"*Q4_K_M*.gguf"           # recommended for GTX 1060 6GB
N_CTX = 2048                         # start safe; later try 4096
N_THREADS = 8                        # set to your CPU threads (e.g. 8/12/16)
N_GPU_LAYERS = -1
matches = glob.glob(str(Path(MODEL_DIR) / PATTERN))

In [3]:
if not matches:
    raise FileNotFoundError(f"No GGUF found in {MODEL_DIR} matching {PATTERN}")



In [7]:
MODEL_PATH = matches[0]
t0 = time.time()
llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=N_CTX,
    n_threads=N_THREADS,
    n_gpu_layers=N_GPU_LAYERS,
    verbose=False,
)
print(f"Loaded in {time.time() - t0:.2f}s")

llama_new_context_with_model: n_ctx_per_seq (2048) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


Loaded in 2.66s


In [8]:
def generate(prompt: str, max_tokens: int = 256, temperature: float = 0.7, top_p: float = 0.95) -> str:
    out = llm(
        prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        repeat_penalty=1.1,
        stop=["</s>", "### Instruction:", "### Response:"],
    )
    return out["choices"][0]["text"].strip()

In [13]:
prompt = """### Instruction:
Какво е кардиотокография в акушерството

### Response:
"""
print(generate(prompt, max_tokens=220))
SYSTEM = "Ти си полезен асистент. Отговаряй на български. Бъди кратък и точен."
_history = []


Кардиотокографията в акушерството е метод за мониторинг на сърдечната дейност на плода по време на бременност. Тя се извършва чрез трансвагинално предаване на електроди, които регистрират електрическите сигнали от сърцето на плода и ги преобразуват в акустични звуци, които могат да бъдат чути с помощта на стетоскоп или ултразвуков монитор.

Кардиотокографията предоставя важна информация за здравето на плода и може да помогне на медицинските специалисти да идентифицират потенциални проблеми още в ранните етапи на бременността. Тя е особено полезна по време на бременност с повишен риск, като например при високорискова бременност или преждевременно раждане.


In [ ]:
def chat(user_msg: str, keep_history: int = 6) -> str:
    _history.append(("Потребител", user_msg))
    _history[:] = _history[-keep_history:]

    hist_text = "\n".join([f"{role}: {msg}" for role, msg in _history])

    prompt = f"""### Instruction:
{SYSTEM}

{hist_text}

### Response:
"""
    reply = generate(prompt, max_tokens=512, temperature=0.6, top_p=0.95)
    _history.append(("Асистент", reply))
    _history[:] = _history[-keep_history:]
    return reply

In [ ]:
print(chat("Здравей! Как можеш да ми помогнеш днес?"))
